In [1]:
import numpy as np
import pandas as pd
import os 
import glob
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pprint
import pygsheets

## Updates to Make if Possible

1. Try Automatically Downloading Any of the data sources. (Chase & Citi would help tremendously!) 
2. Automatically delete rows that are at the date cutoff



In [2]:
order = ['Source','Transaction Date', 'Description', 'Category',  'Amount',
        'Month','Year','Type', 'Exclude', 'Only Exclusions', 'Without Exclusions']
def file_names(file):
    if file.find('Checking') == 0:
        return 'Checking'
    if file.find('Chase') == 0:
        return 'Chase'
    if file.find('Market') == 0:
        return 'Savings'
    if file.find('From') == 0:
        return 'Citi'
    if file.find('CreditCard')==0:
        return 'Wells Credit Card'
    else:
        return 'N/A'    

def checking_exclusions(description):
    if (description.find('CITI CARD')==0) \
        or (description.find('Acorns')==0) \
        or (description.find('CHASE CREDIT')==0) \
        or (description.find('ONLINE PAYMENT')==0):
        return 'x'
    elif (description.find('ONLINE TRANSFER')==0) \
        or (description.find('RECURRING TRANSFER')==0):
        return 'N'
    else:
        return ''

    
def compile_files( file , file_name ):
    
    if file_name == 'Chase':
        cur_data = pd.read_csv(file)
        cur_data.drop( columns = ['Post Date'],inplace= True )
        cur_data['Exclude'] = cur_data['Type'].replace({'Payment': 'x', 'Sale': ''})
        cur_data['Type'] = cur_data['Type'].replace({'Payment':'Income','Sale':'Spent'})
   
    if (file_name == 'Checking') or (file_name == 'Savings') or (file_name == 'Wells Credit Card'):
        cur_data = pd.read_csv(file, header =None)
        cur_data.drop(columns= [2,3], inplace=True)
        cur_data.columns = ['Transaction Date','Amount','Description']
        cur_data['Type'] = cur_data.apply(lambda x: 'Income' if x['Amount'] > 0 else 'Spent', axis =1 )
        
        cur_data['Exclude'] = cur_data['Description'].apply( lambda x: checking_exclusions(x) )
        
        #Need to Do some Pythong String Searching 
        cur_data['Category']  = ''
    
    if file_name == 'Citi':
        cur_data = pd.read_csv(file)
        cur_data.rename(columns = {'Date':'Transaction Date'}, inplace =True)
        cur_data['Amount'] = cur_data.apply( lambda x: -1*np.nansum( [x['Debit'], x['Credit'] ])  ,axis = 1 )
        cur_data['Type'] = cur_data.apply(lambda x: 'Income' if x['Amount'] > 0 else 'Spent', axis =1 )
        cur_data['Exclude'] = cur_data['Type'].replace({'Income': 'x', 'Spent': ''})
        cur_data['Category']  = ''
        cur_data.drop(columns= ['Debit','Credit'], inplace = True)
        
    cur_data['Month'] =  pd.DatetimeIndex(cur_data['Transaction Date']).month
    cur_data['Year'] =  pd.DatetimeIndex(cur_data['Transaction Date']).year
    cur_data['Type'] = cur_data['Type'].replace({'Return': 'Income'})
    cur_data['Source'] = file_name
    cur_data['Only Exclusions'] =  cur_data.apply(lambda x: float(x['Amount']) if (x['Exclude']== 'x') or (x['Exclude']== 'N')  else 0, axis =1)
    cur_data['Without Exclusions'] =  cur_data.apply(lambda x: float(x['Amount']) if (x['Exclude']!= 'x') and (x['Exclude']!= 'N') else 0, axis =1)     
      
    return cur_data[order]

In [3]:
path = "/Users/egordon/Documents/Coding/budget/data/"
os.chdir(path)
files = glob.glob("*.csv") + glob.glob("*.CSV")
date_cutoff = input("Input date Cutoff (mm/dd/yyyy): ")
date_cutoff =  pd.to_datetime(date_cutoff)

Input date Cutoff (mm/dd/yyyy): 01/01/2021


In [4]:
# Chase [3:4]
all_data = []
for file in files:  #files[1:4]: 
    print(file_names(file))
    all_data.append(compile_files( file, file_names(file)))
    
compiled_data = pd.concat(all_data)

#Weirdly Wells Fargo was Downloading Earlier Data, so cut it off here
compiled_data['Transaction Date'] = pd.to_datetime(compiled_data['Transaction Date'])
compiled_data = compiled_data[compiled_data['Transaction Date'] >= date_cutoff]

#Sort for ease of Checking on Google Sheet
compiled_data.sort_values('Transaction Date', inplace =True)

#compiled_data = compiled_data
print("Min Date: ",min(compiled_data['Transaction Date']), "Transaction Records: ", len(compiled_data))

compiled_data.head()

Savings
Checking
Chase
Citi
Min Date:  2021-01-01 00:00:00 Transaction Records:  75


,Source,Transaction Date,Description,Category,Amount,Month,Year,Type,Exclude,Only Exclusions,Without Exclusions
22,Citi,2021-01-01,CHICK-FIL-A #03688 ROHNERT PARK CA,,-24.27,1,2021,Spent,,0.0,-24.27
25,Chase,2021-01-01,SUB 70 GOLF,Shopping,-1.99,1,2021,Spent,,0.0,-1.99
26,Chase,2021-01-01,PAYPAL *DISNEY PLUS,Entertainment,-7.00,1,2021,Spent,,0.0,-7.00
21,Citi,2021-01-02,PAYPAL *SIMPLEHUMAN 4029357733 CA Digital Acco...,,-56.84,1,2021,Spent,,0.0,-56.84
21,Chase,2021-01-02,TCB*MTA METER MTA I,Travel,-0.50,1,2021,Spent,,0.0,-0.50


In [6]:
os.chdir("/Users/egordon/Documents/Coding/budget/")
gc = pygsheets.authorize(service_file = 'Budget Wrangling-e5de1c7718f7.json')

wkbook = gc.open('Budget')

#select the "Data Sheet"
sheet = wkbook[1]

#Get the max Row number with the data to date. 
max_indx = len(sheet.get_col(1, include_tailing_empty = False))

#update the first sheet with df, starting at cell B2. 
print("Will write new data starting on the", str(max_indx + 1) +"th" , "row")

Will write new data starting on the 2982th row


In [7]:
try:
    sheet.set_dataframe(compiled_data,( max_indx + 1 ,1 ), copy_head =False)
    print("Success Writing Data!")
except:
    print("Error Writing the Data!: \n")
    sheet.set_dataframe(compiled_data,( max_indx + 1 ,1 ), copy_head =False)

Success Writing Data!
